In [16]:
import torch
import clip
import numpy as np
import pandas as pd
import json

In [38]:
def model_setup(model):
    """Initial loading of CLIP model."""

    available_models = clip.available_models()

    if model in available_models:
        print(f'Loading model: {model}')
        chosen_model = model
    else:
        print(f'{model} unavailable! Using default model: ViT-L/14@336px')
        chosen_model = available_models[0]

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    model, pps = clip.load(chosen_model, device=device, jit=False)

    print(f'Done! Model loaded to {device} device')
    return model, pps


def get_similarities(img, txts):
    """Grab similarity between text and image embeddings."""
    image_features = torch.from_numpy(img).to('cuda')
    similarity = 100.0 * image_features @ txts.T

    return similarity


def get_sims_dict(img_embs, txt_prompts, txt_embs):
    final_dict = {}
    for _, emb in img_embs.iterrows():
        name = emb['file']
        img_features = emb['embeddings']
        img_sims = get_similarities(img_features, txt_embs)
        sims_dict = {}
        for label, score in zip(txt_prompts, img_sims[0]):
            sims_dict[label] = score.cpu().numpy().item()
        final_dict[name] = sims_dict
    return final_dict


def get_top_synm(final_dict):
    files = final_dict.keys()
    wins = []
    for val in final_dict.values():
        scores_list = list(val.values())
        label_list = list(val.keys())
        np_scores = np.asarray(scores_list)
        windex = np.where(np_scores == np_scores.max())[0][0]
        wins.append(label_list[windex])

    top_synm_dict = {'file': files, 'winner': wins}
    return top_synm_dict


def get_sum_synms(final_dict, man_prompts):
    files = final_dict.keys()
    preds = []

    for key, val in final_dict.items():
        man_score = 0
        woman_score = 0
        for k, v in val.items():
            if k in man_prompts:
                man_score += v
            else:
                woman_score += v
        preds.append(f'Male' if man_score > woman_score else f'Female')

    sum_dict = {'file': files, 'preds': preds}
    return (sum_dict)


def synm_to_gender(synm, man_prompts):
    if synm in man_prompts:
        return 'Male'
    else:
        return 'Female'
    

In [72]:
ROOT = "/home/lazye/Documents/ufrgs/mcs/clip/clip-bias-explore/\
fair-face-classification"
LABELS_PATH = ROOT + "/data/labels"
EMBS_PATH = ROOT + "/data/embeddings"

fface_df = pd.read_csv(ROOT+"/data/fface_val.csv")
vit_model = model_setup('ViT-B/16')
img_embs = pd.read_pickle(EMBS_PATH+"/fface_val_img_embs.pkl")
txt_embs = torch.load(EMBS_PATH+"/synms_labels.pt")

with open(LABELS_PATH+"/bots_synms_prompts.json") as f:
    data = json.load(f)

prompt_list = list(data.values())
man_prompts = prompt_list[:10]
woman_prompts = prompt_list[10:]

Loading model: ViT-B/16
Done! Model loaded to cuda device


In [73]:
sims_dict = get_sims_dict(img_embs, prompt_list, txt_embs)

In [77]:
sum_dict = get_sum_synms(sims_dict, man_prompts)
sum_df = pd.DataFrame(data=sum_dict)
print(sum_df.head())

        file   preds
0  val/1.jpg    Male
1  val/2.jpg  Female
2  val/3.jpg    Male
3  val/4.jpg  Female
4  val/5.jpg    Male


In [78]:
top_dict = get_top_synm(final_dict=sims_dict)
top_df = pd.DataFrame(data=top_dict)
print(top_df.head())

        file                        winner
0  val/1.jpg              a photo of a boy
1  val/2.jpg       a photo of an old woman
2  val/3.jpg  a photo of a middle-aged man
3  val/4.jpg    a photo of a female person
4  val/5.jpg      a photo of a male person
